In [1]:
import json
import torch
import pandas as pd
import numpy as np
from collections import Counter
from torch_geometric import nn as geo_nn
from sklearn.metrics import roc_auc_score
import torch.nn as nn
from torch_geometric.loader import DataLoader
from torch.nn import functional as F
from torch_scatter import scatter_mean

#### to load data use:

```python
features_friends = pd.read_csv("data/FINAL_FEATURES_FRIENDS.tsv", sep='\t')
all_seq_traintest = pd.read_csv("data/FINAL_ALL_SEQUENCES_TRAINTEST.tsv", sep='\t')
features_traintest = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv", sep='\t')
target_traintest = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep='\t')
seq_matrix = pd.read_csv("data/FINAL_SEQUENCES_MATRIX.csv", sep='\t')
```

In [2]:
features_traintest = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv", sep='\t')
features_friends = pd.read_csv("data/FINAL_FEATURES_FRIENDS.tsv", sep='\t')

In [3]:
def make_data(x):
    friends = features_friends[features_friends["CLIENT_ID"] == x["CLIENT_ID"]]
    friend_embedding = friends.mean(axis=0)
    usr_features = pd.merge(x.to_frame().T, friend_embedding.to_frame().T, how="left",
                            left_on="CLIENT_ID", right_on="CLIENT_ID")
    return usr_features.drop(["FRIEND_ID"], axis=1).reset_index(drop=True)

friends_dataset = features_traintest.apply(lambda x: make_data(x), axis=1)
#features_friends.groupby(["CLIENT_ID"]).apply(lambda x: make_data(x))
print("ы")
friends_dataset.to_csv("data/user_friends_embeddings.csv")
#friends_dataset = features_traintest.apply(lambda x: make_data(x), axis=1)
#test_friends_dataset = test_target_features.apply(lambda x: make_data(x), axis=1)
#targets = train_target_features["TARGET"].values

#friends_dataset[0].shape
#friends_dataset.to_csv("data/)

ы


In [6]:
friends = pd.concat(friends_dataset.values)

In [8]:
friends.to_csv("data/user_friends_embeddings.csv")

In [106]:
friend_embedder(X_val).mean(axis=0), y_val.float().mean()

(tensor([0.2898, 0.7102], grad_fn=<MeanBackward1>), tensor(0.2951))

In [ ]:
features_friends = pd.read_csv("data/FINAL_FEATURES_FRIENDS.tsv", sep='\t')
all_seq_traintest = pd.read_csv("data/FINAL_ALL_SEQUENCES_TRAINTEST.tsv", sep='\t')
features_traintest = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv", sep='\t')
target_traintest = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep='\t')
seq_matrix = pd.read_csv("data/FINAL_SEQUENCES_MATRIX.csv", sep='\t')

## Word2Vec

In [2]:
from gensim.models import Word2Vec

In [12]:
def get_sentences(name, column):
    sentences = pd.read_csv(name, sep='\t')
    sentences = sentences[column].apply(lambda x: json.loads(x.replace("'", '"')))
    return sentences

sentences_mat = get_sentences("data/FINAL_SEQUENCES_MATRIX.csv", "SESSIONS_SEQUENCES")
sentences_all = get_sentences("data/FINAL_ALL_SEQUENCES_TRAINTEST.tsv", "SEQUENCE")
sentences = np.concatenate([sentences_mat, sentences_all])
sentences.shape[0] == sentences_mat.shape[0] + sentences_all.shape[0]

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [21]:
model = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1, workers=4)
model.save("data/word2vec.model")

In [4]:
model = Word2Vec.load("data/word2vec.model")
#model.train(sentences, total_examples=sentences.shape[0], epochs=5)

## Сумма эмбеддингов -> ctb

In [13]:
def collect_graph(row):
    if row['SEQUENCE'] == 0:
        return np.zeros(200)
    sequence = json.loads(row['SEQUENCE'].replace("'", '"'))
    if len(sequence) >= 1:
        return np.mean([model.wv[word] for word in sequence], axis=0)
    else:
        return np.zeros(200)

target_traintest = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep='\t')
# test_ids  = target_traintest[target_traintest["TARGET"] == "test"].index
#train_ids = target_traintest[target_traintest["TARGET"] != "test"].index
all_seq_traintest = pd.read_csv("data/FINAL_ALL_SEQUENCES_TRAINTEST.tsv", sep='\t')

targeted_sequences = pd.merge(target_traintest, all_seq_traintest, on="CLIENT_ID", how='left')
targeted_sequences.fillna(0, inplace=True)
graph_sequences = np.vstack(targeted_sequences.apply(lambda x: collect_graph(x), axis=1))
#graph_sequences["CLIENT_ID"] = targeted_sequences["CLIENT_ID"]
#graph_sequences["TARGET"] = targeted_sequences["TARGET"]
graph_sequences.shape

KeyboardInterrupt: 

In [34]:
meanval = y_train.mean()
class_weights = [meanval, 1 - meanval]
class_weights

[0.2741932952476615, 0.7258067047523384]

In [14]:
np.random.seed(42)

In [26]:
np.save("data/graph_sequences.npy", graph_sequences)

In [35]:
graphs = np.load("data/graph_sequences.npy")
graphs.shape

(159290, 200)

In [47]:
X_train = graphs[target_traintest["TARGET"] != "test"]
X_test  = graphs[target_traintest["TARGET"] == "test"]
y_train = target_traintest[target_traintest["TARGET"] != "test"]["TARGET"].apply(int).values

In [37]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

mask = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=False)
ratio = int(0.8 * len(mask))
#targets = targeted_sequences["TARGET"].values

ctb_graph = CatBoostClassifier(iterations=1400, depth=5, learning_rate=1e-3, class_weights=class_weights)
ctb_graph.fit(X_train[mask[:ratio]], y_train[mask[:ratio]], verbose=False)

ctb_graph.save_model("shredder67/catboosts/ctb_graph.ctb")

In [49]:
ctb_graph = CatBoostClassifier(iterations=1400, depth=5, learning_rate=1e-3, class_weights=class_weights)
ctb_graph.fit(X_train, y_train, verbose=False)

ctb_graph.save_model("shredder67/catboosts/ctb_graph_full.ctb")

In [38]:
preds = ctb_graph.predict_proba(X_train[mask[ratio:]])
roc_auc_score(y_train[mask[ratio:]], preds[:, 1])

0.5657667437769949

In [18]:
ctb_graph.save_model("shredder67/catboosts/ctb_graph.ctb")

In [39]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
friends_users = pd.read_csv("data/user_friends_embeddings.csv")
friends_users = friends_users.drop(["Unnamed: 0"], axis=1)

In [40]:
target_traintest = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep='\t')
friends_users_target = pd.merge(target_traintest, friends_users, on="CLIENT_ID", how="left")
friends_users_target

,CLIENT_ID,RETRO_DT,TARGET,i1047_x,i1048_x,i1056_x,i1058_x,i1059_x,i1060_x,i1065_x,...,u8=17_y,u8=18_y,u8=19_y,u8=2_y,u8=20_y,u8=21_y,u8=6_y,u8=7_y,u8=8_y,u8=9_y
0,1011725,20210501,0,0.0,0.0,304.0,0.0,0.0,28.0,165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018784,20210501,1,0.0,3.0,157.0,0.0,0.0,0.0,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1021812,20210501,0,0.0,250.0,314.0,3.0,12.0,525.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1024003,20210501,0,0.0,4.0,17.0,0.0,3.0,3.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1025140,20210501,test,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159285,72633,20210731,0,0.0,0.0,24.0,0.0,0.0,0.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159286,72634,20210731,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159287,72641,20210731,0,0.0,1.0,20.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159288,72642,20210731,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
train = friends_users_target[friends_users_target["TARGET"] != "test"]
test  = friends_users_target[friends_users_target["TARGET"] == "test"]
X_train, X_test = train.drop(["TARGET"], axis=1), test.drop(["TARGET"], axis=1)
y_train = train["TARGET"].apply(int)

In [43]:
np.random.seed(42)

X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

mask = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=False)
ratio = int(0.8 * len(mask))

X_train, X_val = X_train.iloc[mask[:ratio]], X_train.iloc[mask[ratio:]]
y_train, y_val = y_train.iloc[mask[:ratio]], y_train.iloc[mask[ratio:]]

In [45]:
ctb_friend_features = CatBoostClassifier(iterations=1400, depth=5, 
                                         learning_rate=1e-3, class_weights=class_weights)
ctb_friend_features.fit(X_train, y_train, verbose=False)

preds = ctb_friend_features.predict_proba(X_val)
roc_auc_score(y_val, preds[:, 1])

0.5951540482566186

In [52]:
ctb_friend_features = CatBoostClassifier(iterations=1400, depth=5, 
                                         learning_rate=1e-3, class_weights=class_weights)
ctb_friend_features.fit(train.drop('TARGET', axis=1), y_train, verbose=False, plot=True)

ctb_friend_features.save_model("shredder67/catboosts/ctb_friend_features_full.ctb")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [46]:
ctb_friend_features.save_model("shredder67/catboosts/ctb_friend_features.ctb")

In [21]:
target_traintest[target_traintest["TARGET"] == 'test']

,CLIENT_ID,RETRO_DT,TARGET
4,1025140,20210501,test
8,1029732,20210501,test
18,1079794,20210501,test
23,1116331,20210501,test
27,1136822,20210501,test
...,...,...,...
159263,70294,20210731,test
159276,71247,20210731,test
159278,71657,20210731,test
159284,72631,20210731,test


In [20]:
pd.read_csv("sample_submission.csv")

,CLIENT_ID,TARGET
0,1504364,0.623702
1,1592092,0.754212
2,1582159,0.756855
3,1431753,0.880515
4,1571196,0.094763
...,...,...
31853,1518155,0.100278
31854,1544546,0.643005
31855,1584674,0.314064
31856,1733854,0.147863


In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score


grid_params = {"depth": [3, 5, 6, 7],
               "iterations": [300, 500, 700, 1000, 1300],
               "learning_rate": [1e-3, 1e-4, 3e-5]}

ctb_graph = CatBoostClassifier()
ctb_graph.grid_search(param_grid=grid_params,
                      X = X_train,
                      y = y_train,
                      plot=True,
                      verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6929152	test: 0.6929264	best: 0.6929264 (0)	total: 18ms	remaining: 5.39s
1:	learn: 0.6926994	test: 0.6927048	best: 0.6927048 (1)	total: 42.8ms	remaining: 6.38s
2:	learn: 0.6924804	test: 0.6924863	best: 0.6924863 (2)	total: 64ms	remaining: 6.33s
3:	learn: 0.6922568	test: 0.6922645	best: 0.6922645 (3)	total: 87.4ms	remaining: 6.47s
4:	learn: 0.6920374	test: 0.6920473	best: 0.6920473 (4)	total: 108ms	remaining: 6.4s
5:	learn: 0.6918121	test: 0.6918307	best: 0.6918307 (5)	total: 130ms	remaining: 6.35s
6:	learn: 0.6915908	test: 0.6916124	best: 0.6916124 (6)	total: 155ms	remaining: 6.48s
7:	learn: 0.6913765	test: 0.6913942	best: 0.6913942 (7)	total: 176ms	remaining: 6.44s
8:	learn: 0.6911547	test: 0.6911757	best: 0.6911757 (8)	total: 194ms	remaining: 6.26s
9:	learn: 0.6909358	test: 0.6909603	best: 0.6909603 (9)	total: 216ms	remaining: 6.26s
10:	learn: 0.6907189	test: 0.6907457	best: 0.6907457 (10)	total: 234ms	remaining: 6.15s
11:	learn: 0.6905012	test: 0.6905312	best: 0.6905312 

## Разделяем на 2 бустика

In [2]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [3]:
features_traintest = pd.read_csv("data/FINAL_FEATURES_TRAINTEST.tsv", sep='\t')
friends_users = pd.read_csv("data/user_friends_embeddings.csv")
friends_users = friends_users.drop(["Unnamed: 0"], axis=1)

In [4]:
target_traintest = pd.read_csv("data/FINAL_TARGETS_DATES_TRAINTEST.tsv", sep='\t')
friends_users_target = pd.merge(target_traintest, friends_users, on="CLIENT_ID", how="left")
friends_users_target.fillna(0, inplace=True)
friends_users_target.head()

,CLIENT_ID,RETRO_DT,TARGET,i1047_x,i1048_x,i1056_x,i1058_x,i1059_x,i1060_x,i1065_x,...,u8=17_y,u8=18_y,u8=19_y,u8=2_y,u8=20_y,u8=21_y,u8=6_y,u8=7_y,u8=8_y,u8=9_y
0,1011725,20210501,0,0.0,0.0,304.0,0.0,0.0,28.0,165.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1018784,20210501,1,0.0,3.0,157.0,0.0,0.0,0.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1021812,20210501,0,0.0,250.0,314.0,3.0,12.0,525.0,107.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1024003,20210501,0,0.0,4.0,17.0,0.0,3.0,3.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1025140,20210501,test,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
columns = [col for col in friends_users_target.columns if col[-2:] == "_x"]
features_friends_target = friends_users_target.drop(columns, axis=1)
features_user_target = friends_users_target[["CLIENT_ID", "RETRO_DT", "TARGET"] + columns]

In [6]:
features_friends_target.head()

,CLIENT_ID,RETRO_DT,TARGET,i1047_y,i1048_y,i1056_y,i1058_y,i1059_y,i1060_y,i1065_y,...,u8=17_y,u8=18_y,u8=19_y,u8=2_y,u8=20_y,u8=21_y,u8=6_y,u8=7_y,u8=8_y,u8=9_y
0,1011725,20210501,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1018784,20210501,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1021812,20210501,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1024003,20210501,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1025140,20210501,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
features_user_target.head()

,CLIENT_ID,RETRO_DT,TARGET,i1047_x,i1048_x,i1056_x,i1058_x,i1059_x,i1060_x,i1065_x,...,u8=17_x,u8=18_x,u8=19_x,u8=2_x,u8=20_x,u8=21_x,u8=6_x,u8=7_x,u8=8_x,u8=9_x
0,1011725,20210501,0,0.0,0.0,304.0,0.0,0.0,28.0,165.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1018784,20210501,1,0.0,3.0,157.0,0.0,0.0,0.0,92.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1021812,20210501,0,0.0,250.0,314.0,3.0,12.0,525.0,107.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1024003,20210501,0,0.0,4.0,17.0,0.0,3.0,3.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1025140,20210501,test,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
np.random.seed(42)

meanval = target_traintest[target_traintest["TARGET"] != "test"]["TARGET"].mean()
class_weights = [meanval, 1 - meanval]

params = {"iterations": 1400, "depth": 5, 
          "learning_rate": 1e-3, "class_weights": class_weights,
          "verbose": False}

train = features_user_target[features_user_target["TARGET"] != 'test'].reset_index(drop=True)
test = features_user_target[features_user_target["TARGET"] == 'test'].drop("TARGET", axis=1).reset_index(drop=True)
X_train, y_train = train.drop("TARGET", axis=1), train["TARGET"].astype(int).values

mask = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=False)
ratio = int(len(mask) * 0.8)

X_train, X_val = X_train.iloc[mask[:ratio]], X_train.iloc[mask[ratio:]]
y_train, y_val = y_train[mask[:ratio]], y_train[mask[ratio:]]

ctb_users = CatBoostClassifier(class_weights=class_weights, iterations=1400, learning_rate=1e-3)
#ctb_users.set_params(**params)
ctb_users.fit(X_train, y_train)
preds = ctb_users.predict_proba(X_val)
print(roc_auc_score(y_val, preds[:, 1]))

Learning rate set to 0.085016
0:	learn: 0.4458527	total: 40.6ms	remaining: 40.5s
1:	learn: 0.4453566	total: 83.6ms	remaining: 41.7s
2:	learn: 0.4448696	total: 128ms	remaining: 42.5s
3:	learn: 0.4445109	total: 165ms	remaining: 41s
4:	learn: 0.4441420	total: 205ms	remaining: 40.7s
5:	learn: 0.4438422	total: 240ms	remaining: 39.7s
6:	learn: 0.4436097	total: 274ms	remaining: 38.9s
7:	learn: 0.4433031	total: 315ms	remaining: 39s
8:	learn: 0.4430539	total: 355ms	remaining: 39.1s
9:	learn: 0.4428425	total: 409ms	remaining: 40.5s
10:	learn: 0.4426209	total: 453ms	remaining: 40.7s
11:	learn: 0.4424735	total: 499ms	remaining: 41.1s
12:	learn: 0.4423109	total: 549ms	remaining: 41.7s
13:	learn: 0.4421486	total: 600ms	remaining: 42.3s
14:	learn: 0.4420408	total: 636ms	remaining: 41.8s
15:	learn: 0.4419305	total: 682ms	remaining: 41.9s
16:	learn: 0.4417500	total: 716ms	remaining: 41.4s
17:	learn: 0.4416448	total: 749ms	remaining: 40.9s
18:	learn: 0.4415250	total: 796ms	remaining: 41.1s
19:	learn: 0.

In [55]:
def train_ctb(data, parameters, fit_all=False):
    X_test = data[data["TARGET"] == 'test'].drop("TARGET", axis=1).reset_index(drop=True)
    train = data[data["TARGET"] != 'test'].reset_index(drop=True)
    X_train, y_train = train.drop("TARGET", axis=1), train["TARGET"].astype(int).values

    ctb = CatBoostClassifier(**parameters)
    if fit_all:
        ctb.fit(X_train, y_train, verbose=False)
        preds = ctb.predict_proba(X_test)
        return ctb
    np.random.seed(42)
    mask = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=False)
    ratio = int(0.8 * len(mask))

    X_train, X_val = X_train.iloc[mask[:ratio]], X_train.iloc[mask[ratio:]]
    y_train, y_val = y_train.iloc[mask[:ratio]], y_train.iloc[mask[ratio:]]
    
    ctb.fit(X_train, y_train, verbose=False)
    preds = ctb.predict_proba(X_val)
    print(roc_auc_score(y_val, preds[:, 1]))
    return ctb

In [56]:
meanval = target_traintest[target_traintest["TARGET"] != "test"]["TARGET"].mean()
class_weights = [meanval, 1 - meanval]
class_weights

params = {"iterations": 1400, "depth": 5, 
          "learning_rate": 1e-3, "class_weights": class_weights}
ctb_users = train_ctb(features_user_target, params, fit_all=True)
params = {"iterations": 400, "depth": 3, 
          "learning_rate": 3e-4, "class_weights": class_weights}
ctb_friends = train_ctb(features_friends_target, params, fit_all=True)

Learning rate set to 0.088067
0:	learn: 0.4454708	total: 63.6ms	remaining: 1m 3s
1:	learn: 0.4449514	total: 113ms	remaining: 56.4s
2:	learn: 0.4444076	total: 165ms	remaining: 54.7s
3:	learn: 0.4440239	total: 225ms	remaining: 56s
4:	learn: 0.4436214	total: 295ms	remaining: 58.7s
5:	learn: 0.4432616	total: 343ms	remaining: 56.8s
6:	learn: 0.4429646	total: 394ms	remaining: 55.9s
7:	learn: 0.4426267	total: 453ms	remaining: 56.2s
8:	learn: 0.4423798	total: 515ms	remaining: 56.7s
9:	learn: 0.4421386	total: 572ms	remaining: 56.6s
10:	learn: 0.4419616	total: 618ms	remaining: 55.5s
11:	learn: 0.4418150	total: 669ms	remaining: 55.1s
12:	learn: 0.4416493	total: 722ms	remaining: 54.8s
13:	learn: 0.4414924	total: 776ms	remaining: 54.7s
14:	learn: 0.4413387	total: 825ms	remaining: 54.2s
15:	learn: 0.4411985	total: 875ms	remaining: 53.8s
16:	learn: 0.4410937	total: 920ms	remaining: 53.2s
17:	learn: 0.4409848	total: 973ms	remaining: 53.1s
18:	learn: 0.4408662	total: 1.04s	remaining: 53.7s
19:	learn: 0

In [57]:
ctb_users.save_model("shredder67/catboosts/ctb_user_features.ctb")
ctb_friends.save_model("shredder67/catboosts/ctb_friends_features.ctb")

In [19]:
features_user_target.to_csv("data/features_user_target.csv", index=False)
features_friends_target.to_csv("data/features_friends_target.csv", index=False)